In [51]:
# Import Libraries
import requests
import pandas as pd
import numpy as np
import folium

In [52]:
# Constants
LAT_LOWER = 31.3334
LAT_UPPER = 36.9989
LONG_LOWER = -114.8141
LONG_UPPER = -109.0451

API_ENCODING = "Basic [Your Encoding Here]" # Add your Encoded for use token found on https://wigle.net/account here

In [53]:
def searchSSID(query, map) :
    # Builds search query request url
    urlBase = "https://api.wigle.net/api/v2/network/search?"
    urlParams = "&latrange1=" + str(LAT_LOWER) + "&latrange2=" + str(LAT_UPPER) + "&longrange1=" + str(LONG_LOWER) + "&longrange2=" + str(LONG_UPPER) + "&ssid="
    query = query.replace(" ", "%20")
    urlWithQuery = urlBase + urlParams + query
    
    # Initate GET Request
    response = requests.get(urlWithQuery, headers = {'Authorization' : API_ENCODING})

    if response.status_code == 200 :
        results = response.json().get("results", [])
        trilats = [result.get("trilat") for result in results]
        trilongs = [result.get("trilong") for result in results]

        for i in range(len(trilats)) :
            plotPoints([trilats[i],trilongs[i]], map)
    else:
        print("Failed \nMessage: %s" %response.json()['message'])

In [54]:
# Adds poits to interactive map
def plotPoints(coords, map) :
    folium.CircleMarker(coords, 3).add_to(map)
    

In [55]:
# Create interactive map
map = folium.Map(location=[(LAT_LOWER+LAT_UPPER)/2,(LONG_LOWER+LONG_UPPER)/2], zoom_start=7)

searchSSID("McDonalds", map)

map.save("WigleMap.html")